In [ ]:
import json
import re
import numpy as np
import pandas as pd
from pyvi import ViTokenizer

# Các thư viện Machine Learning
from sklearn_crfsuite import CRF
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Các thư viện Deep Learning
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.utils import to_categorical

In [ ]:
import json
import os

def process_data_json(input_path, output_path):
    try:
        # Kiểm tra xem file có tồn tại không
        if not os.path.exists(input_path):
            print(f"Lỗi: Không tìm thấy file tại đường dẫn: {input_path}")
            return

        # 1. Đọc dữ liệu từ đường dẫn data/train02.json
        with open(input_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Kiểm tra cấu trúc dữ liệu
        if not isinstance(data, list):
            print("Lỗi: Cấu trúc JSON phải là một danh sách các bài viết.")
            return

        processed_data = []
        
        # 2. Lấy 15 bài đầu tiên (hoặc ít hơn nếu file không đủ 15 bài)
        # Đánh số ID tự động từ 31 đến 45
        for i, item in enumerate(data[:15], start=31):
            new_item = {
                "id": i,
                "text": item.get("text", ""),
                "label": item.get("label", [])
            }
            processed_data.append(new_item)

        # 3. Xuất file kết quả
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(processed_data, f, ensure_ascii=False, indent=2)
        
        print(f"--- Hoàn tất ---")
        print(f"Đã xử lý xong {len(processed_data)} bài.")
        print(f"File mới đã được lưu tại: {output_path}")

    except json.JSONDecodeError:
        print("Lỗi: File JSON không đúng định dạng.")
    except Exception as e:
        print(f"Có lỗi xảy ra: {e}")

# --- THỰC THI ---
# Đường dẫn bạn cung cấp
input_file = 'data/train03.json' 
output_file = 'data/train03_cleaned.json'

process_data_json(input_file, output_file)

--- Hoàn tất ---
Đã xử lý xong 15 bài.
File mới đã được lưu tại: data/train03_cleaned.json


In [ ]:
import json
import os

def merge_json_files(file_list, output_filename):
    merged_data = []
    current_id = 1

    for file_path in file_list:
        if not os.path.exists(file_path):
            print(f"Cảnh báo: Không tìm thấy file {file_path}. Bỏ qua...")
            continue
            
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                
                # Đảm bảo data là một danh sách
                if isinstance(data, list):
                    for item in data:
                        # Xử lý trường hợp tên trường bị swap giữa 'text' và 'content'
                        content_text = item.get("text") or item.get("content") or ""
                        
                        # Tạo object mới chỉ giữ lại các trường cần thiết
                        clean_item = {
                            "id": current_id,
                            "content": content_text,
                            "label": item.get("label", [])
                        }
                        merged_data.append(clean_item)
                        current_id += 1
                
            except json.JSONDecodeError:
                print(f"Lỗi: File {file_path} không đúng định dạng JSON.")

    # Ghi ra file train.json
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, ensure_ascii=False, indent=2)
    
    print(f"--- THÀNH CÔNG ---")
    print(f"Đã gộp xong {len(merged_data)} bài vào file: {output_filename}")

# --- CẤU HÌNH ĐƯỜNG DẪN ---
# Bạn hãy thay đổi tên file đúng với thực tế của bạn ở đây
danh_sach_file = [
    'data/train01_cleaned.json', 
    'data/train02_cleaned.json', 
    'data/train03_cleaned.json',
    'data/train04_cleaned.json'
]
file_ket_qua = 'train.json'

merge_json_files(danh_sach_file, file_ket_qua)

--- THÀNH CÔNG ---
Đã gộp xong 59 bài vào file: train.json


In [ ]:
import json
import re
from pyvi import ViTokenizer

# --- HÀM 1: CẮT CÂU & TÍNH LẠI OFFSET ---
def split_sentences_and_realign_labels(raw_data):
    split_data = []
    new_id_counter = 1

    for item in raw_data:
        original_text = item.get('text') or item.get('content')
        original_labels = item.get('label', [])
        
        # Tách câu
        sentences = re.split(r'(?<=[.!?])\s+', original_text)
        
        current_cursor = 0
        for sent in sentences:
            sent = sent.strip()
            if not sent: continue
            
            # Tìm vị trí
            start_index = original_text.find(sent, current_cursor)
            end_index = start_index + len(sent)
            current_cursor = end_index
            
            # Tính lại Label
            sent_labels = []
            for lbl in original_labels:
                l_start = lbl['start']
                l_end = lbl['end']
                
                if l_start >= start_index and l_end <= end_index:
                    new_label = {
                        "start": l_start - start_index,
                        "end": l_end - start_index,
                        "text": lbl['text'],
                        "labels": lbl['labels']
                    }
                    sent_labels.append(new_label)
            
            # --- ĐOẠN NÀY ĐÃ ĐƯỢC SỬA ---
            # Chỉ thêm vào danh sách nếu câu có nội dung VÀ có ít nhất 1 label
            if sent and sent_labels:  # <--- QUAN TRỌNG: Lọc bỏ câu không có label
                new_entry = {
                    "id": new_id_counter,
                    "original_id": item.get('id'),
                    "text": sent,
                    "label": sent_labels
                }
                split_data.append(new_entry)
                new_id_counter += 1
                
    return split_data

# --- HÀM 2: LÀM SẠCH & CHUYỂN BIO ---
def convert_to_bio_with_cleaning(json_data):
    all_sentences = []
    for item in json_data:
        original_text = item.get('text') or item.get('content')
        labels = item.get('label', [])
        
        # Tokenize
        tokenized_text = ViTokenizer.tokenize(original_text)
        tokens = tokenized_text.split()
        
        bio_tags = []
        for token in tokens:
            # Clean token
            clean_t = re.sub(r'[^\w\s\d_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂÂÊÔƠƯưăâêôơư]', '', token)
            if not clean_t: continue 
            
            raw_word = clean_t.replace('_', ' ')
            
            tag = "O"
            for lbl in labels:
                lbl_text = lbl['text'].strip()
                if raw_word in lbl_text: 
                    l_type = lbl['labels'][0]
                    if lbl_text.startswith(raw_word):
                        tag = f"B-{l_type}"
                    else:
                        tag = f"I-{l_type}"
                    break
            bio_tags.append((clean_t, tag))
        all_sentences.append(bio_tags)
    return all_sentences


Đang đọc file train.json...
-> Đã đọc 59 văn bản dài.
Đang cắt nhỏ câu và tính lại vị trí nhãn...
-> Đã lưu 368 câu ngắn vào file: train_short.json
Đang chuyển đổi sang định dạng BIO...
-> Đã lưu dữ liệu BIO vào file: train_bio.json

=== HOÀN TẤT ===


In [ ]:
input_file = 'train.json' # Tên file gốc của bạn

try:
    # 1. Load dữ liệu gốc
    print(f"1. Đang đọc file gốc: {input_file}...")
    with open(input_file, 'r', encoding='utf-8') as f:
        raw_long_data = json.load(f)
    print(f"   -> Tìm thấy {len(raw_long_data)} văn bản gốc.")

    # 2. Cắt nhỏ thành từng câu & Lọc bỏ câu rác (không label)
    print("2. Đang cắt câu và lọc bỏ câu không có nhãn...")
    short_data = split_sentences_and_realign_labels(raw_long_data)
    
    if len(short_data) == 0:
        print("   ⚠️ CẢNH BÁO: Không có câu nào được giữ lại! Kiểm tra xem file gốc có label hợp lệ không.")
    else:
        print(f"   -> Kết quả: Giữ lại được {len(short_data)} câu chất lượng (có chứa thực thể).")

    # --- LƯU FILE 1: DATA ĐÃ CẮT NHỎ (Dạng JSON chuẩn) ---
    output_short = 'train_short.json'
    with open(output_short, 'w', encoding='utf-8') as f:
        json.dump(short_data, f, ensure_ascii=False, indent=2)
    print(f"   -> Đã lưu file sạch vào: {output_short}")

    # 3. Làm sạch ký tự đặc biệt & Chuyển sang BIO
    print("3. Đang chuyển đổi sang định dạng BIO để train...")
    # Lưu ý: Cần đảm bảo bạn đã khai báo hàm convert_to_bio_with_cleaning ở trên rồi nhé
    train_sents = convert_to_bio_with_cleaning(short_data)
    
    # --- LƯU FILE 2: DATA BIO ---
    output_bio = 'train_bio.json'
    with open(output_bio, 'w', encoding='utf-8') as f:
        json.dump(train_sents, f, ensure_ascii=False, indent=2)
    print(f"   -> Đã lưu dữ liệu BIO vào: {output_bio}")
    
    print("\n=== ✅ HOÀN TẤT QUY TRÌNH TIỀN XỬ LÝ ===")

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{input_file}'. Hãy kiểm tra lại tên file.")
except Exception as e:
    print(f"❌ Có lỗi xảy ra: {e}")

1. Đang đọc file gốc: train.json...
   -> Tìm thấy 59 văn bản gốc.
2. Đang cắt câu và lọc bỏ câu không có nhãn...
   -> Kết quả: Giữ lại được 368 câu chất lượng (có chứa thực thể).
   -> Đã lưu file sạch vào: train_short.json
3. Đang chuyển đổi sang định dạng BIO để train...
   -> Đã lưu dữ liệu BIO vào: train_bio.json

=== ✅ HOÀN TẤT QUY TRÌNH TIỀN XỬ LÝ ===


In [ ]:
print(f"Số lượng văn bản gốc: {len(raw_long_data)}")
print(f"Số lượng câu sau khi cắt: {len(short_data)}")

Số lượng văn bản gốc: 59
Số lượng câu sau khi cắt: 368


In [ ]:
short_data

{'id': 11,
 'original_id': 2,
 'text': 'Lúc đó, Chủ tịch Hồ Chí Minh từ trần, vì Tố Hữu đang ốm nằm viện nên Trung ương Đảng Lao động Việt Nam không thông báo cho ông.',
 'label': [{'start': 8,
   'end': 28,
   'text': 'Chủ tịch Hồ Chí Minh',
   'labels': ['PER']},
  {'start': 41, 'end': 47, 'text': 'Tố Hữu', 'labels': ['PER']},
  {'start': 69,
   'end': 102,
   'text': 'Trung ương Đảng Lao động Việt Nam',
   'labels': ['ORG']}]}

In [ ]:
def get_avg_len(data):
    # Use 'text' if available, otherwise 'content'. Handle empty lists to avoid ZeroDivisionError.
    lengths = [len(item.get('text') or item.get('content', '')) for item in data]
    if not lengths:
        return 0
    return sum(lengths) / len(lengths)

len_old = get_avg_len(raw_long_data)
len_new = get_avg_len(short_data)

print(f"Độ dài trung bình 1 bài gốc: {len_old:.0f} ký tự")
print(f"Độ dài trung bình 1 câu mới: {len_new:.0f} ký tự")

Độ dài trung bình 1 bài gốc: 779 ký tự
Độ dài trung bình 1 câu mới: 124 ký tự


In [ ]:
import json
import re
from pyvi import ViTokenizer

# --- HÀM 1: CẮT CÂU & TÍNH LẠI OFFSET ---
def split_sentences_and_realign_labels(raw_data):
    split_data = []
    new_id_counter = 1
    
    # --- BIẾN THỐNG KÊ ---
    total_detected = 0   # Tổng số câu tách được từ regex
    total_removed = 0    # Số câu bị xóa vì không có label
    
    for item in raw_data:
        original_text = item.get('text') or item.get('content')
        original_labels = item.get('label', [])
        
        # Tách câu
        sentences = re.split(r'(?<=[.!?])\s+', original_text)
        
        current_cursor = 0
        for sent in sentences:
            sent = sent.strip()
            if not sent: continue
            
            total_detected += 1 # Đếm câu tìm thấy
            
            # Tìm vị trí
            start_index = original_text.find(sent, current_cursor)
            end_index = start_index + len(sent)
            current_cursor = end_index
            
            # Tính lại Label
            sent_labels = []
            for lbl in original_labels:
                l_start = lbl['start']
                l_end = lbl['end']
                
                if l_start >= start_index and l_end <= end_index:
                    new_label = {
                        "start": l_start - start_index,
                        "end": l_end - start_index,
                        "text": lbl['text'],
                        "labels": lbl['labels']
                    }
                    sent_labels.append(new_label)
            
            # ĐIỀU KIỆN LỌC
            if sent and sent_labels: 
                new_entry = {
                    "id": new_id_counter,
                    "original_id": item.get('id'),
                    "text": sent,
                    "label": sent_labels
                }
                split_data.append(new_entry)
                new_id_counter += 1
            else:
                total_removed += 1 # Tăng biến đếm nếu bị xóa
                
    return split_data

# --- HÀM 2: LÀM SẠCH & CHUYỂN BIO ---
def convert_to_bio_with_cleaning(json_data):
    all_sentences = []
    for item in json_data:
        original_text = item.get('text') or item.get('content')
        labels = item.get('label', [])
        
        # Tokenize
        tokenized_text = ViTokenizer.tokenize(original_text)
        tokens = tokenized_text.split()
        
        bio_tags = []
        for token in tokens:
            # Clean token
            clean_t = re.sub(r'[^\w\s\d_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂÂÊÔƠƯưăâêôơư]', '', token)
            if not clean_t: continue 
            
            raw_word = clean_t.replace('_', ' ')
            
            tag = "O"
            for lbl in labels:
                lbl_text = lbl['text'].strip()
                if raw_word in lbl_text: 
                    l_type = lbl['labels'][0]
                    if lbl_text.startswith(raw_word):
                        tag = f"B-{l_type}"
                    else:
                        tag = f"I-{l_type}"
                    break
            bio_tags.append((clean_t, tag))
        all_sentences.append(bio_tags)
    return all_sentences

# =======================================================
# PHẦN THỰC THI CHÍNH (MAIN)
# =======================================================
input_file = 'train.json' 

try:
    with open(input_file, 'r', encoding='utf-8') as f:
        raw_long_data = json.load(f)
    
    # Gọi hàm để xem kết quả số lượng
    short_data = split_sentences_and_realign_labels(raw_long_data)

    # Lưu file kết quả
    with open('train_short.json', 'w', encoding='utf-8') as f:
        json.dump(short_data, f, ensure_ascii=False, indent=2)

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {input_file}")


       BÁO CÁO XỬ LÝ DỮ LIỆU
1. Tổng số câu tách ra từ văn bản gốc:  368
2. Số câu bị xóa (do không có label): - 120
----------------------------------------
3. SỐ CÂU CÒN LẠI (DÙNG ĐỂ TRAIN):    = 248

